In [6]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import warnings
import tools
from scipy.constants import c, nu2lambda

In [7]:
from IPython.core.display import HTML
from IPython.core.pylabtools import figsize
from IPython.display import display, Math

HTML("""
<style>
.output_png{
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [8]:
figsize(8,4)

In [9]:
'''sys.path.append('C:\\Program Files\\Lumerical\\v221\\api\\python')
sys.path.append('C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py')'''
sys.path.append('C:\\Program Files\\Lumerical\\v242\\api\\python')
sys.path.append('C:\\Program Files\\Lumerical\\v242\\api\\python\\lumapi.py')

In [10]:
warnings.filterwarnings("ignore", category=SyntaxWarning)
import lumapi
fdtdApi = lumapi.FDTD()

In [11]:
fdtdApi.switchtolayout()
fdtdApi.deleteall()

In [12]:
#units
nm = 1e-9
um = 1e-6

In [13]:
# Parametros do acoplador direcional
Lc = np.linspace(0,45*um,16)

gap = 200*nm
rad = 5*um
# Parametros do guia de onda
wg_width = 500*nm
wg_height = 220*nm
# Materiais do guia de onda
material_Clad = 'SiO2 (Glass) - Palik'
material_core = 'Si (Silicon) - Palik'

In [14]:
#Banda de operação 
wl_start = 1500*nm
wl_stop = 1600*nm
nbmFrqPnts = 21
# Nome do arquivo
filename = 'dc_FDTd_%d_nm_%03d_um'%(gap/nm,Lc[0]/um) 

In [16]:
## Configuração do FDTD
# Tempo de simulação
simulation_time = (np.pi*rad+Lc[0])*7/c+200e-15

# Dimensões da porta
portSize = 2*um
portBcDist = 6*um
# Dimensões do volume do solver
x_FDTD = Lc[0]/2 + 2*rad
y_FDTD = -rad-gap/2-wg_height
x_span_FDTD = 1.1*Lc[0]+2*rad+portBcDist
y_span_FDTD = 2*rad + gap + 2*wg_width+portSize
z_span_FDTD = 5*um
# Material
backGroundMaterial = material_Clad
# Precisão do mesh
mesh_accuracy = 3
# condições de fronteira
z_min_bc = 'PML'

In [17]:
# Parametrosa das portas
y_port_off = 0.5*um
x_port = np.array([Lc[0]+3*rad, rad, Lc[0]+3*rad, rad])
y_port = np.array([-(2*rad+gap+wg_width+y_port_off),-(2*rad+gap+wg_width+y_port_off),0+y_port_off,0+y_port_off])
x_span_port = 4*um
z_span_port = 4*um
mode_selection = 'fundamental TE mode'
injection_axis = 'y-axis'

## Criando dispositivo

In [40]:
gap = 200 * nm
gap2 = 300 * nm
x_span_core = 500 * nm
x_span_core_top = 600 * nm
x_span_core_bottom = 400 * nm
x_core = gap2 / 2
y_core = 0.0 * um
y_span_core = 0.220 * um
z_core = 0.0 * um
z_core_span = wg_height
def buildDevice(lc):
    for i in range(4):
        fdtdApi.addobject('90_bend_wg')
        fdtdApi.set('name',f'arco{i}')
        fdtdApi.set('material',material_core)
        fdtdApi.set('base width',wg_width)
        fdtdApi.set('base height',wg_height)
        fdtdApi.set('start angle', i*90)
        fdtdApi.set('radius',rad)
        fdtdApi.set('z',0)
        
    fdtdApi.setnamed('arco0', 'x', 2*rad + lc)
    fdtdApi.setnamed('arco1', 'x', 2*rad)
    fdtdApi.setnamed('arco2', 'x', 2*rad)
    fdtdApi.setnamed('arco3', 'x', 2*rad + lc)
    
    fdtdApi.setnamed('arco0', 'y', -(gap+wg_width+2*rad))
    fdtdApi.setnamed('arco1', 'y', -(gap+wg_width+2*rad))
    fdtdApi.setnamed('arco2', 'y', 0)
    fdtdApi.setnamed('arco3', 'y', 0)
    
    for i in range(4):
        fdtdApi.addrect()
        fdtdApi.set('name',f'port{i}')
        fdtdApi.set('material',material_core)
        fdtdApi.set('x span',wg_width)
        fdtdApi.set('y span',portSize)
        fdtdApi.set('z span',wg_height)
        fdtdApi.set('z',0)
        
    fdtdApi.setnamed('port0', 'y', -(portSize/2 + 2*rad+gap+wg_width))
    fdtdApi.setnamed('port0', 'x', lc+3*rad)
    fdtdApi.setnamed('port1', 'y', -(portSize/2 + 2*rad+gap+wg_width))
    fdtdApi.setnamed('port1', 'x', rad)
    
    fdtdApi.setnamed('port2', 'y', portSize/2)
    fdtdApi.setnamed('port2', 'x', lc+3*rad)
    fdtdApi.setnamed('port3', 'y', portSize/2)
    fdtdApi.setnamed('port3', 'x', rad)
    
    fdtdApi.addrect()
    fdtdApi.set('name','wav01')
    fdtdApi.set('material',material_core)
    fdtdApi.set('x', 2*rad + lc/2)
    fdtdApi.set('x span',lc)
    fdtdApi.set('y', -rad)
    fdtdApi.set('y span',wg_width)
    fdtdApi.set('z',0)
    fdtdApi.set('z span',wg_height)
    
    fdtdApi.addrect()
    fdtdApi.set('name','wav02')
    fdtdApi.set('material',material_core)
    fdtdApi.set('x', 2*rad + lc/2)
    fdtdApi.set('x span',lc)
    fdtdApi.set('y', -rad-gap-wg_width)
    fdtdApi.set('y span',wg_width)
    fdtdApi.set('z',0)
    fdtdApi.set('z span',wg_height)
    return _

## Adição do solver FDTD, porta e monitor

In [41]:
# solver
def SolverPorts(simulation_time):
    fdtdApi.addfdtd()
    fdtdApi.set('x', fdtdApi.getnamed('wav01','x'))
    fdtdApi.set('x span', fdtdApi.getnamed('wav01','x span') + 12*um)
    fdtdApi.set('y', y_FDTD)
    fdtdApi.set('y span', y_span_FDTD)
    fdtdApi.set('z', 0)
    fdtdApi.set('z span', z_span_FDTD)
    fdtdApi.set('background material', backGroundMaterial)
    fdtdApi.set('mesh accuracy', mesh_accuracy)
    fdtdApi.set('simulation time', simulation_time)
    fdtdApi.set('z min bc', z_min_bc)
    
    # portas
    for i in range(4):
        fdtdApi.addport()
        fdtdApi.set('name',f'port {i+1}')
        fdtdApi.set('x',fdtdApi.getnamed(f'port{i}','x'))
        fdtdApi.set('x span',x_span_port)
        fdtdApi.set('y',y_port[i])
        fdtdApi.set('z',0)
        fdtdApi.set('z span',z_span_port)
        fdtdApi.set('mode selection',mode_selection)
        fdtdApi.set('injection axis',injection_axis)
    for i in range(4):
        fdtdApi.select(f'FDTD::ports::port {i+1}')
        fdtdApi.set('x span', 4*1e-6)
    fdtdApi.setnamed('FDTD::ports::port 3','direction','Backward')
    fdtdApi.setnamed('FDTD::ports::port 4','direction','Backward')
    
    # configuração da banda de simulação
    fdtdApi.setnamed('FDTD','global source wavelength start',wl_start)
    fdtdApi.setnamed('FDTD','global source wavelength stop',wl_stop)
    fdtdApi.setnamed('FDTD::ports','monitor frequency points',nbmFrqPnts)
    
    return _

## S parameters Sweep

In [42]:
def runSweep():
    
    fdtdApi.addsweep(3)
    fdtdApi.setsweep('s-parameter sweep', 'name', 'sweep')
    fdtdApi.setsweep('sweep', 'Excite all ports', 1)
    fdtdApi.setsweep('sweep', 'Calculate group delay', 1)
    fdtdApi.setsweep('sweep', 'Custom define', 1)
    
    fdtdApi.setsweep('sweep', 'define custom s-parameter ports', 1)
    fdtdApi.setsweep('sweep', 'Include group delay', 1)
    
    fdtdApi.setsweep('sweep', 'Custom define', 1)
    #ports locations 
    modestruct = {"label": "mode 1", "id" : 1}
    rowstruct = {"mode 1": modestruct, "location": "BOTTOM"}
    rowstruct1 = {"mode 1": modestruct, "location": "TOP"}
    portstruct0 = {"port 1": rowstruct}
    portstruct = {"port 2": rowstruct} 
    portstruct2 = {"port 3": rowstruct1}
    portstruct3 = {"port 4": rowstruct1}
    fdtdApi.setsweep("sweep", "export setup", portstruct0)
    fdtdApi.setsweep("sweep", "export setup", portstruct)
    fdtdApi.setsweep("sweep", "export setup", portstruct2)
    fdtdApi.setsweep("sweep", "export setup", portstruct3)
    
    fdtdApi.setsweep("S sweep", "auto symmetry", True)
    fdtdApi.runsweep('sweep')
    return _

In [43]:
for i in range(0,4,1):
    print(Lc[i]/um, i)
print('---')
for i in range(4,9,1):
    print(Lc[i]/um, i)
print('---')
for i in range(9,13,1):
    print(Lc[i]/um, i)
print('---')
for i in range(13,16,1):
    print(Lc[i]/um, i) 

0.0 0
3.0 1
6.0 2
8.999999999999998 3
---
12.0 4
15.0 5
17.999999999999996 6
21.0 7
24.0 8
---
26.999999999999996 9
30.0 10
33.0 11
35.99999999999999 12
---
38.99999999999999 13
42.0 14
45.0 15


## Dispositivos Lc = 0 ~ 9 $\mu$m

In [44]:
for i in range(0,4,1):
    fdtdApi.switchtolayout()
    fdtdApi.deleteall()
    buildDevice(Lc[i])
    fdtdApi.select('FDTD')
    fdtdApi.delete()
    simulation_time = (np.pi*rad+Lc[i])*7/c+200e-15
    SolverPorts(simulation_time)
    # salvando arquivo
    filename ='dc_FDTD_%d_nm_%03d_um'%(gap*nm,Lc[i]*1e7)
    fdtdApi.save(filename)
    fdtdApi.select('FDTD')
    fdtdApi.set('express mode',1)
    runSweep()
    fdtdApi.exportsweep('sweep',f'S_param_{round(Lc[i]/um)}um.dat')

## Dispositivos Lc = 12 ~ 24 $\mu$m 

In [45]:
for i in range(4,9,1):
    fdtdApi.switchtolayout()
    fdtdApi.deleteall()
    buildDevice(Lc[i])
    fdtdApi.select('FDTD')
    fdtdApi.delete()
    simulation_time = (np.pi*rad+Lc[i])*7/c+200e-15
    SolverPorts(simulation_time)
    # salvando arquivo
    filename ='dc_FDTD_%d_nm_%03d_um'%(gap*nm,Lc[i]*1e7)
    fdtdApi.save(filename)
    fdtdApi.select('FDTD')
    fdtdApi.set('express mode',1)
    runSweep()
    fdtdApi.exportsweep('sweep',f'S_param_{round(Lc[i]/um)}um.dat')

## Dispositivos Lc = 27 ~ 36 $\mu$m

In [46]:
for i in range(9,13,1):
    fdtdApi.switchtolayout()
    fdtdApi.deleteall()
    buildDevice(Lc[i])
    fdtdApi.select('FDTD')
    fdtdApi.delete()
    simulation_time = (np.pi*rad+Lc[i])*7/c+200e-15
    SolverPorts(simulation_time)
    # salvando arquivo
    filename ='dc_FDTD_%d_nm_%03d_um'%(gap*nm,Lc[i]*1e7)
    fdtdApi.save(filename)
    fdtdApi.select('FDTD')
    fdtdApi.set('express mode',1)
    runSweep()
    fdtdApi.exportsweep('sweep',f'S_param_{round(Lc[i]/um)}um.dat')

## Dispositivos Lc = 39 ~ 45 $\mu$m

In [47]:
for i in range(13,16,1):
    fdtdApi.switchtolayout()
    fdtdApi.deleteall()
    buildDevice(Lc[i])
    fdtdApi.select('FDTD')
    fdtdApi.delete()
    simulation_time = (np.pi*rad+Lc[i])*7/c+200e-15
    SolverPorts(simulation_time)
    # salvando arquivo
    filename ='dc_FDTD_%d_nm_%03d_um'%(gap*nm,Lc[i]*1e7)
    fdtdApi.save(filename)
    fdtdApi.select('FDTD')
    fdtdApi.set('express mode',1)
    runSweep()
    fdtdApi.exportsweep('sweep',f'S_param_{round(Lc[i]/um)}um.dat')

In [48]:
'''
f = gdsopen('meugds.gds');
cellname = 'contour';

gdsbegincell(f,cellname);
for(i =1:length(contour{1}.polygons)){
    a = contour{1}.polygons{i};
    gdsaddpoly(f,1,a);  
}
gdsendcell(f);
gdsclose(f);
'''

"\nf = gdsopen('meugds.gds');\ncellname = 'contour';\n\ngdsbegincell(f,cellname);\nfor(i =1:length(contour{1}.polygons)){\n    a = contour{1}.polygons{i};\n    gdsaddpoly(f,1,a);  \n}\ngdsendcell(f);\ngdsclose(f);\n"